In [ ]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
###########assign memory##########
###########delete this part if your tensorflow was based on CPU##########
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
###########import packages##########
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.models import Sequential 
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
%matplotlib
###########fix random seed for reproducability##########
seed=2
np.random.seed(seed)
###########loading data##########
fdata=pd.read_csv('database.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'BET Surface Area cm2/g',
                      'Micropore ratio',
                      'Mesopore ratio',
                      'Macropore ratio',
                      'Membrane Thickness',
                      'I/C Ratio(ionomer/catalyst)',
                      'Anodic Platinum Loading Amount mgPt cm-2',
                      'Anodic catalyst type x wt% Pt/C',
                      'Hot Press Temperature ℃',
                      'Hot Press Time min',
                      'Hot Press Pressure Mpa',
                      'Humidity %',
                      'Operating Temperature ℃',
                      'Flowing rate of H2 ml min-1 ',
                      'Flowing rate of O2 ml min-1 ',
                      'Back Pressure Mpa',
                      'Cathodic Loading Amount mg cm-2',
                      'Co Cotent wt/%',
                      'Fe Cotent wt/%(ICP for metal)',
                      'N Content at/%',
                      'O Content at/%',
                      'S Content at/%',
                      'Pyridinic N+metal N Content ratio at/%',
                      'Pyrrolic N Content ratio at/%',
                      'Graphitic N Content ratio at/%',
                      'Oxidized N Content ratio at/%',#26features
                      'Open Circuit Voltage V',
                      'Voltage 100mA cm0',
                      'Voltage 200mA cm0',
                      'Voltage 300mA cm0',
                      'Voltage 400mA cm0',
                      'Voltage 500mA cm0',
                      'Voltage 600mA cm0',
                      'Voltage 700mA cm0',
                      'Voltage 800mA cm0',
                      'Voltage 900mA cm0',
                      'Voltage 1000mA cm0',
                      'Voltage 1100mA cm0',
                      'Voltage 1200mA cm0',
                      'Voltage 1300mA cm0',
                      'Voltage 1400mA cm0',
                      'Voltage 1500mA cm0',
                      'Voltage 1600mA cm0',
                      'Voltage 1700mA cm0',
                      'Voltage 1800mA cm0',
                      'Voltage 1900mA cm0',
                      'Voltage 2000mA cm0',
                      'Voltage 2100mA cm0',
                      'Voltage 2200mA cm0',
                      'Voltage 2300mA cm0',
                      'Voltage 2400mA cm0',
                      'Voltage 2500mA cm0',
                      'Voltage 2600mA cm0',
                      'Voltage 2700mA cm0',
                      'Voltage 2800mA cm0']]#29 outputs
###########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
###########data standardization##########
raw_power=data.iloc[:,26:]
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)
param=standardized_data.iloc[:,0:26]
power=standardized_data.iloc[:,26:]
###########randomly pick 4 curves to drop out from the database as testset##########
param=param.drop(index=[13,19,62,33])
power=power.drop(index=[13,19,62,33])
X=param.values.astype(np.float32)
y=power.values.astype(np.float32)
###########keras ANN model construction##########
neurons1=1800
activation1='softsign'
regularizer=keras.regularizers.l2(0.00001)
dropout_rate=0.8
model = Sequential() 
model.add(Dense(neurons1, input_dim=26, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(29, input_dim=neurons1, activation='linear'))
adam=optimizers.Adam(lr=0.001)
model.compile(loss='mse', optimizer=adam) 
print('Training ------------')
###########train the model with the training set##########
###########testset has been removed from X and y before##########
model.fit(X, y, epochs=1250, batch_size=120,validation_split=0.15,verbose=1,callbacks=[TensorBoard(log_dir='mytensorboard-curve')])
###########create a vector with 26 features as later input##########
###########here we simply use the median and replace the values by user inputs##########
median=standardized_data.median()
raw_median=data.median()
###########the 62th curve in database-testset##########
user_input=raw_median[0:26]
# BET Surface Area cm2/g
user_input.iloc[0]=579
# Micropore ratio
user_input.iloc[1]=0.1297
# Mesopore ratio
user_input.iloc[2]=0.8703
# Macropore ratio
user_input.iloc[3]=0
#Membrane thickness
user_input.iloc[4]=25
# I/C Ratio(ionomer/catalyst)
user_input.iloc[5]=1
# Anodic Platinum Loading Amount mgPt cm-2
user_input.iloc[6]=0.2
# Anodic catalyst type x wt% Pt/C
user_input.iloc[7]=0.2
# Hot Press Temperature ℃
user_input.iloc[8]=130
# Hot Press Time min
user_input.iloc[9]=1.5
# Hot Press Pressure Mpa
user_input.iloc[10]=1.5
# Humidity % 
user_input.iloc[11]=100
# Operating Temperature ℃ 
user_input.iloc[12]=80
# Flowing rate of H2 ml min-1 
user_input.iloc[13]=300
# Flowing rate of O2 ml min-1
user_input.iloc[14]=400
# Back Pressure Mpa 
user_input.iloc[15]=0.2
# Cathodic Loading Amount mg cm-2
user_input.iloc[16]=2
# Co Cotent wt/% 
user_input.iloc[17]=0
# Fe Cotent wt/%(ICP for metal)
user_input.iloc[18]=0.02634
# N Content at/%
user_input.iloc[19]=0.1125
# O Content at/%
user_input.iloc[20]=0.0663
# S Content at/% 
user_input.iloc[21]=0
# Pyridinic N+metal N Content ratio at/%
user_input.iloc[22]=0.315702884
# Pyrrolic N Content at/% 
user_input.iloc[23]=0.319148936
# Graphitic N Content at/% 
user_input.iloc[24]=0.36514818
# Oxidized N Content at/%
user_input.iloc[25]=0
###########standardize the user input##########
standardized_user_input=(user_input-np.mean(data,axis=0)[0:26])/np.std(data,axis=0)[0:26]
###########consult the ANN with standardized input##########
data_test_input=standardized_user_input
data_test_input=pd.DataFrame(data_test_input)
data_test_input=data_test_input.T
data_test_param=data_test_input.values.astype(np.float32)
###########get the prediction##########
predict_ann= model.predict(data_test_param)
###########transfer the prediction to voltage values##########
voltage_result_list=np.std(raw_power,axis=0).T.values*predict_ann+np.mean(raw_power,axis=0).T.values
###########computing output power##########
x_current=np.arange(0,2900,100)
predict_power=x_current*voltage_result_list
print(predict_power)
predict_power=predict_power.T
###########real values in literature##########
curve_real=data.iloc[62,26:]
predict_power_real=x_current*data.iloc[62,26:].values
###########computing R2##########
real_current_series=pd.Series(data.iloc[62,26:].values)
voltage_result_series=pd.Series(voltage_result_list[0,:])
corr_ann = round(voltage_result_series.corr(real_current_series), 4)
print('ANN',corr_ann)
###########visualization##########
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_current,voltage_result_list,color='red',label='Predicted Voltage')
ax.plot(x_current,data.iloc[62,26:].values,color='orange')
ax.set_xlabel(u"Current(mA cm^-2)")
ax.set_ylabel(u"Voltage(V)")
ax2 = ax.twinx()
ax2.set_ylabel(u'Power Density (mW cm^-2)')
ax2.scatter(x_current,predict_power,color='green',label='Predicted Power Density')
ax2.plot(x_current,predict_power_real,color='blue')
print(voltage_result_list)
print(data.iloc[62,26:].values)
print(predict_power)
print(predict_power_real)
plt.show()

In [ ]:
###########the 33th curve in database-testset##########
# BET Surface Area cm2/g
user_input.iloc[0]=1265
# Micropore ratio
user_input.iloc[1]=0.84822
# Mesopore ratio
user_input.iloc[2]=0.10119
# Macropore ratio
user_input.iloc[3]=0.05059
#Membrane thickness
user_input.iloc[4]=125
# I/C Ratio(ionomer/catalyst)
user_input.iloc[5]=0.55
# Anodic Platinum Loading Amount mgPt cm-2
user_input.iloc[6]=0.2
# Anodic catalyst type x wt% Pt/C
user_input.iloc[7]=0.282
# Hot Press Temperature ℃
user_input.iloc[8]=120
# Hot Press Time min
user_input.iloc[9]=5
# Hot Press Pressure Mpa
user_input.iloc[10]=5.443
# Humidity % 
user_input.iloc[11]=100
# Operating Temperature ℃ 
user_input.iloc[12]=80
# Flowing rate of H2 ml min-1 
user_input.iloc[13]=300
# Flowing rate of O2 ml min-1
user_input.iloc[14]=400
# Back Pressure Mpa 
user_input.iloc[15]=0.1378
# Cathodic Loading Amount mg cm-2
user_input.iloc[16]=4
# Co Cotent wt/% 
user_input.iloc[17]=0
# Fe Cotent wt/%(ICP for metal)
user_input.iloc[18]=0.0129
# N Content at/%
user_input.iloc[19]=0.0396
# O Content at/%
user_input.iloc[20]=0.0347
# S Content at/% 
user_input.iloc[21]=0
# Pyridinic N+metal N Content ratio at/%
user_input.iloc[22]=0.381313131
# Pyrrolic N Content at/% 
user_input.iloc[23]=0.204545455
# Graphitic N Content at/% 
user_input.iloc[24]=0.328282828
# Oxidized N Content at/%
user_input.iloc[25]=0.085858586
###########standardize the user input##########
standardized_user_input=(user_input-np.mean(data,axis=0)[0:26])/np.std(data,axis=0)[0:26]
###########consult the ANN with standardized input##########
data_test_input=standardized_user_input
data_test_input=pd.DataFrame(data_test_input)
data_test_input=data_test_input.T
data_test_param=data_test_input.values.astype(np.float32)
###########get the prediction##########
predict_ann= model.predict(data_test_param)
###########transfer the prediction to voltage values##########
voltage_result_list=np.std(raw_power,axis=0).T.values*predict_ann+np.mean(raw_power,axis=0).T.values
###########computing output power##########
x_current=np.arange(0,2000,100)
predict_power=x_current*voltage_result_list[0,0:20]
print(predict_power)
predict_power=predict_power.T
###########real values in literature##########
curve_real=data.iloc[33,26:46]
predict_power_real=x_current*data.iloc[33,26:46].values
###########computing R2##########
real_current_series=pd.Series(data.iloc[33,26:46].values)
voltage_result_series=pd.Series(voltage_result_list[0,0:20])
corr_ann = round(voltage_result_series.corr(real_current_series), 4)
print('ANN',corr_ann)
###########visualization##########
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_current,voltage_result_list[0,0:20],color='red',label='Predicted Voltage')
ax.plot(x_current,data.iloc[33,26:46].values,color='orange')
ax.set_xlabel(u"Current(mA cm^-2)")
ax.set_ylabel(u"Voltage(V)")
ax2 = ax.twinx()
ax2.set_ylabel(u'Power Density (mW cm^-2)')
ax2.scatter(x_current,predict_power,color='green',label='Predicted Power Density')
ax2.plot(x_current,predict_power_real,color='blue')
print(voltage_result_list)
print(data.iloc[33,26:].values)
print(predict_power)
print(predict_power_real)
plt.show()

In [ ]:
###########the 13th curve in database-testset##########
# BET Surface Area cm2/g
user_input.iloc[0]=1277
# Micropore ratio
user_input.iloc[1]=1
# Mesopore ratio
user_input.iloc[2]=0
# Macropore ratio
user_input.iloc[3]=0
#Membrane thickness
user_input.iloc[4]=50
# I/C Ratio(ionomer/catalyst)
user_input.iloc[5]=1
# Anodic Platinum Loading Amount mgPt cm-2
user_input.iloc[6]=0.25
# Anodic catalyst type x wt% Pt/C
user_input.iloc[7]=0.2
# Hot Press Temperature ℃
user_input.iloc[8]=120
# Hot Press Time min
user_input.iloc[9]=2
# Hot Press Pressure Mpa
user_input.iloc[10]=9.0718
# Humidity % 
user_input.iloc[11]=100
# Operating Temperature ℃ 
user_input.iloc[12]=80
# Flowing rate of H2 ml min-1 
user_input.iloc[13]=300
# Flowing rate of O2 ml min-1
user_input.iloc[14]=300
# Back Pressure Mpa 
user_input.iloc[15]=0.2
# Cathodic Loading Amount mg cm-2
user_input.iloc[16]=2.2
# Co Cotent wt/% 
user_input.iloc[17]=0
# Fe Cotent wt/%(ICP for metal)
user_input.iloc[18]=0.013
# N Content at/%
user_input.iloc[19]=0.02067
# O Content at/%
user_input.iloc[20]=0.17144
# S Content at/% 
user_input.iloc[21]=0
# Pyridinic N+metal N Content ratio at/%
user_input.iloc[22]=0.535847395
# Pyrrolic N Content at/% 
user_input.iloc[23]=0.275129833
# Graphitic N Content at/% 
user_input.iloc[24]=0.120989201
# Oxidized N Content at/%
user_input.iloc[25]=0.068033571
###########standardize the user input##########
standardized_user_input=(user_input-np.mean(data,axis=0)[0:26])/np.std(data,axis=0)[0:26]
###########consult the ANN with standardized input##########
data_test_input=standardized_user_input
data_test_input=pd.DataFrame(data_test_input)
data_test_input=data_test_input.T
data_test_param=data_test_input.values.astype(np.float32)
###########get the prediction##########
predict_ann= model.predict(data_test_param)
###########transfer the prediction to voltage values##########
voltage_result_list=np.std(raw_power,axis=0).T.values*predict_ann+np.mean(raw_power,axis=0).T.values
###########computing output power##########
x_current=np.arange(0,1500,100)
predict_power=x_current*voltage_result_list[0,0:15]
print(predict_power)
predict_power=predict_power.T
###########real values in literature##########
curve_real=data.iloc[13,26:41]
predict_power_real=x_current*data.iloc[13,26:41].values
###########computing R2##########
real_current_series=pd.Series(data.iloc[13,26:41].values)
voltage_result_series=pd.Series(voltage_result_list[0,0:15])
corr_ann = round(voltage_result_series.corr(real_current_series), 4)
print('ANN',corr_ann)
###########visualization##########
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_current,voltage_result_list[0,0:15],color='red',label='Predicted Voltage')
ax.plot(x_current,data.iloc[13,26:41].values,color='orange')
ax.set_xlabel(u"Current(mA cm^-2)")
ax.set_ylabel(u"Voltage(V)")
ax2 = ax.twinx()
ax2.set_ylabel(u'Power Density (mW cm^-2)')
ax2.scatter(x_current,predict_power,color='green',label='Predicted Power Density')
ax2.plot(x_current,predict_power_real,color='blue')
print(voltage_result_list)
print(data.iloc[13,26:].values)
print(predict_power)
print(predict_power_real)
plt.show()

In [ ]:
###########the 19th curve in database-testset##########
# BET Surface Area cm2/g
user_input.iloc[0]=634
# Micropore ratio
user_input.iloc[1]=0.06322
# Mesopore ratio
user_input.iloc[2]=0.53874
# Macropore ratio
user_input.iloc[3]=0.39804
#Membrane thickness
user_input.iloc[4]=25
# I/C Ratio(ionomer/catalyst)
user_input.iloc[5]=1.5
# Anodic Platinum Loading Amount mgPt cm-2
user_input.iloc[6]=0.6
# Anodic catalyst type x wt% Pt/C
user_input.iloc[7]=0.6
# Hot Press Temperature ℃
user_input.iloc[8]=140
# Hot Press Time min
user_input.iloc[9]=4
# Hot Press Pressure Mpa
user_input.iloc[10]=4
# Humidity % 
user_input.iloc[11]=100
# Operating Temperature ℃ 
user_input.iloc[12]=80
# Flowing rate of H2 ml min-1 
user_input.iloc[13]=300
# Flowing rate of O2 ml min-1
user_input.iloc[14]=300
# Back Pressure Mpa 
user_input.iloc[15]=0
# Cathodic Loading Amount mg cm-2
user_input.iloc[16]=2
# Co Cotent wt/% 
user_input.iloc[17]=0
# Fe Cotent wt/%(ICP for metal)
user_input.iloc[18]=0.013
# N Content at/%
user_input.iloc[19]=0.032
# O Content at/%
user_input.iloc[20]=0.1143
# S Content at/% 
user_input.iloc[21]=0
# Pyridinic N+metal N Content ratio at/%
user_input.iloc[22]=0.294117647
# Pyrrolic N Content at/% 
user_input.iloc[23]=0.205882353
# Graphitic N Content at/% 
user_input.iloc[24]=0.323529412
# Oxidized N Content at/%
user_input.iloc[25]=0.176470588
###########standardize the user input##########
standardized_user_input=(user_input-np.mean(data,axis=0)[0:26])/np.std(data,axis=0)[0:26]
###########consult the ANN with standardized input##########
data_test_input=standardized_user_input
data_test_input=pd.DataFrame(data_test_input)
data_test_input=data_test_input.T
data_test_param=data_test_input.values.astype(np.float32)
###########get the prediction##########
predict_ann= model.predict(data_test_param)
###########transfer the prediction to voltage values##########
voltage_result_list=np.std(raw_power,axis=0).T.values*predict_ann+np.mean(raw_power,axis=0).T.values
###########computing output power##########
x_current=np.arange(0,1500,100)
predict_power=x_current*voltage_result_list[0,0:15]
print(predict_power)
predict_power=predict_power.T
###########real values in literature##########
curve_real=data.iloc[19,26:41]
predict_power_real=x_current*data.iloc[19,26:41].values
###########computing R2##########
real_current_series=pd.Series(data.iloc[19,26:41].values)
voltage_result_series=pd.Series(voltage_result_list[0,0:15])
corr_ann = round(voltage_result_series.corr(real_current_series), 4)
print('ANN',corr_ann)
###########visualization##########
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_current,voltage_result_list[0,0:15],color='red',label='Predicted Voltage')
ax.plot(x_current,data.iloc[19,26:41].values,color='orange')
ax.set_xlabel(u"Current(mA cm^-2)")
ax.set_ylabel(u"Voltage(V)")
ax2 = ax.twinx()
ax2.set_ylabel(u'Power Density (mW cm^-2)')
ax2.scatter(x_current,predict_power,color='green',label='Predicted Power Density')
ax2.plot(x_current,predict_power_real,color='blue')
print(voltage_result_list)
print(data.iloc[19,26:].values)
print(predict_power)
print(predict_power_real)
plt.show()

In [ ]:
9958 48 73 15